Create a notebook or python script named advanced_dataframes to do your work in for these exercises. [check]

Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine) [check]

cd into your exercises folder for this module and run echo env.py >> .gitignore [check]


### Create a function named get_db_url.   
    
It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

 #### Use your function to obtain a connection to the employees database.

In [178]:
from env import get_db_url

import pandas as pd
import numpy as np 


In [193]:
url = get_db_url("employees")


# f"
# format string (f-string) allows us to evaluate variables inside of curly braces
# real world application = makes writing strings with variables easier 



### Once you have successfully run a query:

a. Intentionally make a typo in the database url. What kind of error message do you see?

b. Intentionally make an error in your SQL query. What does the error message look like?



In [189]:
#a 
pd.read_sql('''SELECT * FROM employees LIMIT 5 OFFSET 50''', )

query27 = '''
SELECT * FROM employees
'''

pd.read_sql(query, )

# TypeError: read_sql() missing 1 required positional argument: 'con'

#b

pd.read_sql('''SELECT * FROM employees LIMIT 5 OFFSET 50''', url)

query = '''
SELECT * FROM employee LIMIT 10
'''

pd.read_sql(query, url)

# ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'employees.employee' doesn't exist")
# [SQL: SELECT * FROM employee LIMIT 10]
# (Background on this error at: https://sqlalche.me/e/14/f405)

TypeError: read_sql() missing 1 required positional argument: 'con'

In [200]:
query27 = '''
SELECT * FROM employees
'''
empdf = pd.read_sql(query27, url)
empdf

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


### Read the employees and titles tables into two separate DataFrames.

How many rows and columns do you have in each DataFrame? Is that what you expected?


In [191]:
pd.read_sql('employees', url)
# 300024 rows X 6 columns


pd.read_sql('titles', url)
# 443308 rows x 4 columns

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,NaT
1,10002,Staff,1996-08-03,NaT
2,10003,Senior Engineer,1995-12-03,NaT
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,NaT
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,NaT
443305,499998,Senior Staff,1998-12-27,NaT
443306,499998,Staff,1993-12-27,1998-12-27


### Display the summary statistics for each DataFrame.

How many unique titles are in the titles DataFrame?

In [201]:
pd.read_sql('''SELECT DISTINCT title FROM titles''', url)

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


#### c. What is the oldest date in the to_date column?

In [202]:
pd.read_sql('''SELECT MIN(to_date) FROM titles''', url)

,MIN(to_date)
0,1985-03-01


#### d. What is the most recent date in the to_date column?

In [203]:
pd.read_sql('''SELECT MAX(to_date) FROM titles''', url)

,MAX(to_date)
0,9999-01-01


# Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [20]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [21]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. What is the result of using a right join on the DataFrames?

In [23]:
# df.merge default settings for commonly used parameters.

roles.merge(users, how='right', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [221]:
users.merge(roles, how='right', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)

,id,name,role_id
0,1,admin,NaN
1,2,author,NaN
2,3,reviewer,NaN
3,4,commenter,NaN


In [220]:
right_join = users.merge(roles, 
                       left_on = 'role_id',  
                       right_on='id',
                       how='right',
                       indicator=True)
right_join 

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [30]:
roles.merge(users, how='right')
# same result as above, the other parameters are extraneous here. 
users.merge(roles, how='right')
#merging this way creates all NaN in role_id

,id,name,role_id
0,1,admin,NaN
1,2,author,NaN
2,3,reviewer,NaN
3,4,commenter,NaN


3. What is the result of using an outer join on the DataFrames?

In [24]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)



,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [32]:
roles.merge(users, left_on='id', right_on='role_id', how='outer', indicator=True)

,id_x,name_x,id_y,name_y,role_id,_merge
0,1.0,admin,1.0,bob,1.0,both
1,2.0,author,2.0,joe,2.0,both
2,3.0,reviewer,3.0,sally,3.0,both
3,3.0,reviewer,4.0,adam,3.0,both
4,4.0,commenter,NaN,NaN,NaN,left_only
5,NaN,NaN,5.0,jane,NaN,right_only
6,NaN,NaN,6.0,mike,NaN,right_only


4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [35]:
#users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True).drop(columns='id')
# can no longer be joined since 'id' will no longer be found on the axis 

# another approach
users2 = users.drop(columns='role_id')
users2.merge(roles, how='outer').T

#both df are represented in a minimal and disconnected way. 






,0,1,2,3,4,5,6,7,8,9
id,1,2,3,4,5,6,1,2,3,4
name,bob,joe,sally,adam,jane,mike,admin,author,reviewer,commenter


5. Load the mpg dataset from PyDataset.

In [270]:
# Load the dataset and store it in the variable mpg.

from pydataset import data
mpg = data('mpg')
mpg.head()


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


6. Output and read the documentation for the mpg dataset.

In [271]:
data('mpg', show_doc=True) 

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




7. How many rows and columns are in the dataset?

In [272]:
mpg.shape
# 234 x 11

(234, 11)

8. Check out your column names and perform any cleanup you may want on them.

In [273]:
mpg.columns
mpg = mpg.rename(columns={'cty': 'city'})
mpg = mpg.rename(columns={'hwy': 'highway'})

In [274]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


9. Display the summary statistics for the dataset.

In [275]:
mpg.describe()

,displ,year,cyl,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. How many different manufacturers are there?

In [276]:
distinct_manu = mpg['manufacturer'].unique()
distinct_manu

array(['audi', 'chevrolet', 'dodge', 'ford', 'honda', 'hyundai', 'jeep',
       'land rover', 'lincoln', 'mercury', 'nissan', 'pontiac', 'subaru',
       'toyota', 'volkswagen'], dtype=object)

In [277]:
len(distinct_manu)

15

11. How many different models are there?

In [56]:
distinct_model = mpg['model'].unique()
distinct_model

array(['a4', 'a4 quattro', 'a6 quattro', 'c1500 suburban 2wd', 'corvette',
       'k1500 tahoe 4wd', 'malibu', 'caravan 2wd', 'dakota pickup 4wd',
       'durango 4wd', 'ram 1500 pickup 4wd', 'expedition 2wd',
       'explorer 4wd', 'f150 pickup 4wd', 'mustang', 'civic', 'sonata',
       'tiburon', 'grand cherokee 4wd', 'range rover', 'navigator 2wd',
       'mountaineer 4wd', 'altima', 'maxima', 'pathfinder 4wd',
       'grand prix', 'forester awd', 'impreza awd', '4runner 4wd',
       'camry', 'camry solara', 'corolla', 'land cruiser wagon 4wd',
       'toyota tacoma 4wd', 'gti', 'jetta', 'new beetle', 'passat'],
      dtype=object)

In [57]:
len(distinct_model)

38

12. Create a column named mileage_difference like you did in the DataFrames exercises;   
this column should contain the difference between highway and city mileage for each car.

In [58]:
mpg['mileage_difference'] = mpg.highway - mpg.city 
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10


In [64]:
mpg['mileage_difference'] = np.subtract(mpg['highway'], mpg['city'])
mpg['mileage_difference']

# both columns utilize int64 as their dtype.. 
# but I was getting an error when highway and city were set as their own parameters
# when I referenced them through the df itself, this allowed me to use numpy subtract.

1      11
2       8
3      11
4       9
5      10
       ..
230     9
231     8
232    10
233     8
234     9
Name: mileage_difference, Length: 234, dtype: int64

In [65]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


13. Create a column named average_mileage like you did in the DataFrames exercises;   
this is the mean of the city and highway mileage.

In [278]:
mpg['average_mileage'] = mpg[['city', 'highway']].mean(axis=1)
mpg

# mpg['average_mileage'] = (mpg.highway + mpg.city) / 2
# does this make a difference? 

# now I'll try it using numpy 
#mpg['average_mileage'] = np.mean(mpg['highway' 'city'])
#mpg couldn't make it work after a few tries. Moving on for now


,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,22.0


mean(axis=)   
It specifies the axis along which the means are computed. By default axis=0. This is consistent with the numpy.mean usage when axis is specified explicitly (in numpy.mean, axis==None by default, which computes the mean value over the flattened array) , in which axis=0 along the rows (namely, index in pandas), and axis=1 along the columns. For added clarity, one may choose to specify axis='index' (instead of axis=0) or axis='columns' (instead of axis=1).

In [265]:
mpg.drop(columns='average_mileage_v1')

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic,average_mileage_v2,avg_avg_mileage,average_mileage_v3
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True,23.5,20.149573,NaN
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False,25.0,20.149573,NaN
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False,25.5,20.149573,NaN
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True,25.5,20.149573,NaN
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True,21.0,20.149573,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True,23.5,20.149573,NaN
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False,25.0,20.149573,NaN
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True,21.0,20.149573,NaN
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False,22.0,20.149573,NaN


14. Create a new column on the mpg dataset named is_automatic that holds boolean   
values denoting whether the car has an automatic transmission.

In [85]:
bool_trans = mpg.trans.str.contains('auto')
bool_trans

mpg['is_automatic'] = bool_trans
mpg


,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


15. Using the mpg dataset, find out which which manufacturer has  
the best miles per gallon on average?

In [250]:
mpg.groupby('manufacturer').max('average_mileage')
# Volkswagen 
# incorrect. Trying to figure out why. 

# mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False).round(2)

,displ,year,cyl,city,highway,mileage_difference,average_mileage,is_automatic,average_mileage_v1,average_mileage_v2
manufacturer,,,,,,,,,,
audi,4.2,2008,8,21,31,11,25.5,True,25.5,25.5
chevrolet,7.0,2008,8,22,30,11,26.0,True,26.0,26.0
dodge,5.9,2008,8,18,24,7,21.0,True,21.0,21.0
ford,5.4,2008,8,18,26,9,22.0,True,22.0,22.0
honda,2.0,2008,4,28,36,12,30.5,True,30.5,30.5
hyundai,3.3,2008,6,21,31,10,26.0,True,26.0,26.0
jeep,6.1,2008,8,17,22,5,19.5,True,19.5,19.5
land rover,4.6,2008,8,12,18,6,15.0,True,15.0,15.0
lincoln,5.4,2008,8,12,18,6,15.0,True,15.0,15.0


In [260]:
mpg.groupby('manufacturer').average_mileage_v1.mean().sort_values(ascending=False).round(2)

manufacturer
honda         28.50
volkswagen    25.07
hyundai       22.75
subaru        22.43
audi          22.03
toyota        21.72
pontiac       21.70
nissan        21.35
chevrolet     18.45
ford          16.68
mercury       15.62
jeep          15.56
dodge         15.54
lincoln       14.17
land rover    14.00
Name: average_mileage_v1, dtype: float64

In [259]:
mpg.describe()

,displ,year,cyl,city,highway,mileage_difference,average_mileage,average_mileage_v1,average_mileage_v2,avg_avg_mileage,average_mileage_v3
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,2.340000e+02,0.0
mean,3.471795,2003.500000,5.888889,16.858974,23.440171,6.581197,20.149573,20.149573,20.149573,2.014957e+01,NaN
std,1.291959,4.509646,1.611534,4.255946,5.954643,2.262739,5.050290,5.050290,5.050290,1.424132e-14,NaN
min,1.600000,1999.000000,4.000000,9.000000,12.000000,2.000000,10.500000,10.500000,10.500000,2.014957e+01,NaN
25%,2.400000,1999.000000,4.000000,14.000000,18.000000,5.000000,15.500000,15.500000,15.500000,2.014957e+01,NaN
50%,3.300000,2003.500000,6.000000,17.000000,24.000000,7.000000,20.500000,20.500000,20.500000,2.014957e+01,NaN
75%,4.600000,2008.000000,8.000000,19.000000,27.000000,8.000000,23.500000,23.500000,23.500000,2.014957e+01,NaN
max,7.000000,2008.000000,8.000000,35.000000,44.000000,12.000000,39.500000,39.500000,39.500000,2.014957e+01,NaN


39.5

16. Do automatic or manual cars have better miles per gallon?

# Exercises III

1. Use your get_db_url function to help you explore the   
data from the chipotle database.

_originally tried making a pd Data Frame from the SQL Query, but it created many problems
and caused a lot of confusion while slowing me down. Plus, I was essentially
just making an identical data frame. Altogether an impractical and futile approach.

__sql_converter = pd.read_sql(query, burl)

__chipotle = pd.DataFrame(sql_converter, columns = ['id', 'order_id', 'quantity', 'item_name', 'item_price'])
print(chipotle)

#### never again.


In [156]:
burl = get_db_url("chipotle") #burrito url as burl makes me happy. 

query = ''' SELECT * FROM orders'''

pd.read_sql(query, burl)

chip_orders = pd.read_sql(query, burl)

chip_orders

# after seeing the table, item_price should be changed to a float value. 
chip_orders['item_price'] = chip_orders.item_price.str.replace('$', '').astype(float) 


/var/folders/qn/sd1_3x2j4196db03067vql600000gn/T/ipykernel_3947/1875755615.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chip_orders['item_price'] = chip_orders.item_price.str.replace('$', '').astype(float)


In [159]:
chip_orders

#string be goneeee.

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75


2. What is the total price for each order?

In [154]:
chip_orders.groupby('order_id').item_price.sum()

/var/folders/qn/sd1_3x2j4196db03067vql600000gn/T/ipykernel_3947/1275127636.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chip_orders['item_price'] = chip_orders.item_price.str.replace('$', '').astype(float)


order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

###################################################################    
desired adding total price as a column via

#ALTER TABLE table_name
#ADD new_column_name column_definition
#[FIRST | AFTER column_name];

 #FIRST and AFTER are optional statements that tell MySQL the position 
#for the new column in the table. If this parameter is not specified 
#then the new column is added to the end of the table.

abandoned this idea. I'll leave that to the engineers. 
###################################################################

In [160]:
chip_orders

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75


3. What are the most popular 3 items?

In [169]:
chip_orders.groupby('item_name').quantity.sum().nlargest(3, keep='all')

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

4. Which item has produced the most revenue?

In [173]:
chip_orders.groupby('item_name').item_price.sum().nlargest(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

5. Join the employees and titles DataFrames together.

In [182]:
employees = pd.read_sql('employees', url)
employees
# 300024 rows X 6 columns

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [181]:
titles = pd.read_sql('titles', url)
titles
# 443308 rows x 4 columns

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,NaT
1,10002,Staff,1996-08-03,NaT
2,10003,Senior Engineer,1995-12-03,NaT
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,NaT
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,NaT
443305,499998,Senior Staff,1998-12-27,NaT
443306,499998,Staff,1993-12-27,1998-12-27


In [185]:
inneremptit = employees.merge(titles, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
inneremptit

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,NaT
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,NaT
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,NaT
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,NaT
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,NaT
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,NaT
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


6. For each title, find the hire date of the employee   
that was hired most recently with that title.

In [187]:
inneremptit.groupby('title').hire_date.max()

title
Assistant Engineer   1999-12-24
Engineer             2000-01-28
Manager              1992-02-05
Senior Engineer      2000-01-01
Senior Staff         2000-01-13
Staff                2000-01-12
Technique Leader     1999-12-31
Name: hire_date, dtype: datetime64[ns]

In [188]:


queryemp = '''SELECT t.title, MAX(e.hire_date) FROM employees e JOIN titles t ON e.emp_no = t.emp_no GROUP BY t.title ORDER BY t.title;'''

pd.read_sql(queryemp, url)

,title,MAX(e.hire_date)
0,Assistant Engineer,1999-12-24
1,Engineer,2000-01-28
2,Manager,1992-02-05
3,Senior Engineer,2000-01-01
4,Senior Staff,2000-01-13
5,Staff,2000-01-12
6,Technique Leader,1999-12-31


7. Write the code necessary to create a cross tabulation    
of the number of titles by department.    
(Hint: this will involve a combination of SQL code to pull the    
necessary data and python/pandas code to perform the manipulations.)    